In [5]:
from IPython.display import display
import mne
import pickle
import os
import pandas as pd
import numpy as np

from components.dataset import Dataset_Builder
dataset_path = "data"
dataset = Dataset_Builder(dataset_path=dataset_path)\
            .with_sampling_rate(125)\
            .with_marker(start_minute=1, stop_minute=2, segment_second=5)\
            .build()

data, labels, groups = dataset.load_data_all()
data.shape, labels.shape, groups.shape

Found: 55 files
Mean:20.509090909090908, Std:6.149272617066855
Non-stressed:16
Stressed:19
Neutral:20
number of epochs = 12.0


((420, 16, 625), (420,), (420,))

In [2]:
data, labels, groups = dataset.load_data_all()
data.shape, labels.shape, groups.shape

((700, 16, 1875), (700,), (700,))

In [5]:
a,b,c = dataset.load_data('001')
a.shape, b.shape, c.shape

((20, 16, 1875), (20,), (20,))

In [7]:
np.hstack([b,b])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [10]:
dataset.wh

array([False,  True, False, False,  True,  True, False,  True, False,
       False, False, False,  True, False, False,  True, False, False,
       False, False,  True, False, False,  True, False,  True, False,
       False,  True, False, False,  True, False,  True, False, False,
       False,  True, False,  True, False, False, False,  True,  True,
       False, False, False,  True, False, False,  True, False,  True,
       False])

In [5]:
data, labels = dataset.load_data('001')
print(data.shape, labels.shape)

(20, 16, 1875) (20,)


In [6]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [8]:
dataset.groups

array([0, 1, 2, 2, 1, 1, 2, 1, 2, 2, 0, 0, 1, 2, 0, 1, 0, 0, 0, 2, 1, 0,
       2, 1, 0, 1, 0, 2, 1, 2, 0, 1, 2, 1, 0, 2, 2, 1, 2, 1, 0, 0, 2, 1,
       1, 0, 2, 2, 1, 2, 2, 1, 0, 1, 2])

In [9]:
print(dataset.start_sample)
print(dataset.stop_sample)
print(dataset.step_sample)
data[:, dataset.start_sample:dataset.stop_sample].shape

7500
22500
1875


(16, 15000)

In [17]:
data = dataset.load_data('001')
# We skip the first marker becase 
data = np.split(data[:, dataset.start_sample:dataset.stop_sample], dataset._expected_epochs_number, axis=1)
print(data[0].shape)
data = [np.expand_dims(data,axis=0) for data in data]
data = np.vstack(data)
data.shape

(16, 1875)


(8, 16, 1875)

In [ ]:
# [np.array(16,125*segment),...] => len(data.shape[1]/segment)
data = [np.expand_dims(data,axis=0) for data in data]
# [np.array(1,16,125*segment),...] => len(data.shape[1]/segment)
data = np.vstack(data)
# (len(data.shape[1]/segment) , 16 , 125*segment )

assert data.shape[0] == dataset._expected_epochs_number, f"data.shape={data.shape}. Expect the first dimension to be {dataset._expected_epochs_number}"

return data

In [12]:
import numpy as np
a = np.arange(0,200).reshape(2,10,10)
a = a.ravel()
a

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [14]:
a.reshape(2,20,5)

array([[[  0,   1,   2,   3,   4],
        [  5,   6,   7,   8,   9],
        [ 10,  11,  12,  13,  14],
        [ 15,  16,  17,  18,  19],
        [ 20,  21,  22,  23,  24],
        [ 25,  26,  27,  28,  29],
        [ 30,  31,  32,  33,  34],
        [ 35,  36,  37,  38,  39],
        [ 40,  41,  42,  43,  44],
        [ 45,  46,  47,  48,  49],
        [ 50,  51,  52,  53,  54],
        [ 55,  56,  57,  58,  59],
        [ 60,  61,  62,  63,  64],
        [ 65,  66,  67,  68,  69],
        [ 70,  71,  72,  73,  74],
        [ 75,  76,  77,  78,  79],
        [ 80,  81,  82,  83,  84],
        [ 85,  86,  87,  88,  89],
        [ 90,  91,  92,  93,  94],
        [ 95,  96,  97,  98,  99]],

       [[100, 101, 102, 103, 104],
        [105, 106, 107, 108, 109],
        [110, 111, 112, 113, 114],
        [115, 116, 117, 118, 119],
        [120, 121, 122, 123, 124],
        [125, 126, 127, 128, 129],
        [130, 131, 132, 133, 134],
        [135, 136, 137, 138, 139],
        [140, 141,

In [16]:
band_names = ['delta','theta','alpha','beta','gamma','slow','beta_low']
band_names.index('delta')

0

In [3]:
import numpy as np
np.arange(2,16*8,16)

array([  2,   9,  16,  23,  30,  37,  44,  51,  58,  65,  72,  79,  86,
        93, 100, 107, 114, 121])